In [43]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.preprocessing import MinMaxScaler
from torch.utils.data import DataLoader, TensorDataset
import pandas as pd
import torch.nn.functional as F
import numpy as np
import pickle
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error
import xgboost as xgb
import json





In [10]:
torch.cuda.is_available()

False

In [44]:

test_x = pd.read_parquet("../../data/preprocessed/thursdays.pq")
test_y = pd.read_parquet("../../data/preprocessed/thursdays_y.pq")

X = pd.read_parquet('../../data/preprocessed/full_data.pq')
y = pd.read_parquet('../../data/preprocessed/full_data_y.pq')

X1_train = pd.read_parquet('../../data/preprocessed/holdout_1.pq')
X1_test = pd.read_parquet('../../data/preprocessed/thursdays.pq')
y1_train = pd.read_parquet('../../data/preprocessed/holdout_1_y.pq')
y1_test = pd.read_parquet('../../data/preprocessed/thursdays_y.pq')

X2_train = pd.read_parquet('../../data/preprocessed/holdout_2.pq')
X2_test = pd.read_parquet('../../data/preprocessed/tuesdays.pq')
y2_train = pd.read_parquet('../../data/preprocessed/holdout_2_y.pq')
y2_test = pd.read_parquet('../../data/preprocessed/tuesdays_y.pq')

X3_train = pd.read_parquet('../../data/preprocessed/holdout_3.pq')
X3_test = pd.read_parquet('../../data/preprocessed/wednesdays.pq')
y3_train = pd.read_parquet('../../data/preprocessed/holdout_3_y.pq')
y3_test = pd.read_parquet('../../data/preprocessed/wednesdays_y.pq')

print(f"X1 Train Shapes [{X1_train.shape} / {y1_train.shape}]")
print(f"X1 Test Shapes [{X1_test.shape} / {y1_test.shape}]")
print(f"X2 Train Shapes [{X2_train.shape} / {y2_train.shape}]")
print(f"X2 Test Shapes [{X2_test.shape} / {y2_test.shape}]")
print(X1_train.columns)
X1_test.head()

X1 Train Shapes [(478807, 18) / (478807, 1)]
X1 Test Shapes [(273604, 18) / (273604, 1)]
X2 Train Shapes [(478807, 18) / (478807, 1)]
X2 Test Shapes [(273604, 18) / (273604, 1)]
Index(['ProzessData_ActData_AB1_Analogs_DX1_MassPressure',
       'ProzessData_ActData_AB1_Analogs_GY1_MassLevelTank',
       'ProzessData_ActData_AB1_Current_DV1_Scraper',
       'ProzessData_ActData_AB1_Current_DW1_RiserPumpFwd',
       'ProzessData_ActData_AB1_Speed_DV1_Scraper',
       'ProzessData_ActData_AB1_Speed_DW1_RiserPumpFwd',
       'ProzessData_ActData_AB1_Temperature_DP1_MassHeatingStage',
       'ProzessData_ActData_AB1_Temperature_DP1_WaterHeatingStage',
       'ProzessData_ActData_AB1_Temperature_DQ1_MassCoolingStage',
       'ProzessData_ActData_AB1_Temperature_DQ1_WaterCoolingStage',
       'ProzessData_ActData_AB1_Temperature_DR1_WaterMixingStage',
       'ProzessData_ActData_AB1_Temperature_DS1_WaterPipe',
       'ProzessData_ActData_AB1_Temperature_DU1_WaterTank',
       'ProzessData_ActD

,ProzessData_ActData_AB1_Analogs_DX1_MassPressure,ProzessData_ActData_AB1_Analogs_GY1_MassLevelTank,ProzessData_ActData_AB1_Current_DV1_Scraper,ProzessData_ActData_AB1_Current_DW1_RiserPumpFwd,ProzessData_ActData_AB1_Speed_DV1_Scraper,ProzessData_ActData_AB1_Speed_DW1_RiserPumpFwd,ProzessData_ActData_AB1_Temperature_DP1_MassHeatingStage,ProzessData_ActData_AB1_Temperature_DP1_WaterHeatingStage,ProzessData_ActData_AB1_Temperature_DQ1_MassCoolingStage,ProzessData_ActData_AB1_Temperature_DQ1_WaterCoolingStage,ProzessData_ActData_AB1_Temperature_DR1_WaterMixingStage,ProzessData_ActData_AB1_Temperature_DS1_WaterPipe,ProzessData_ActData_AB1_Temperature_DU1_WaterTank,ProzessData_ActData_AB1_Temperature_DX1_MassInfeed,hour,minute,day_time,night_time
2023-03-16 01:00:00+00:00,0.74,64.07,0.0,0.0,0.0,0.0,41.299999,42.000000,41.599998,42.0,42.0,41.849998,42.000000,41.900002,1,0,0,1
2023-03-16 01:00:01+00:00,0.75,64.07,0.0,0.0,0.0,0.0,41.299999,42.099998,41.599998,42.0,42.0,41.849998,41.900002,41.900002,1,0,0,1
2023-03-16 01:00:02+00:00,0.74,64.07,0.0,0.0,0.0,0.0,41.299999,42.099998,41.599998,42.0,42.0,41.849998,41.900002,41.900002,1,0,0,1
2023-03-16 01:00:03+00:00,0.75,64.07,0.0,0.0,0.0,0.0,41.299999,42.099998,41.599998,42.0,42.0,41.849998,41.900002,41.900002,1,0,0,1
2023-03-16 01:00:04+00:00,0.75,64.07,0.0,0.0,0.0,0.0,41.299999,42.099998,41.599998,42.0,42.0,41.849998,41.900002,41.900002,1,0,0,1


In [11]:
len(train_y)

478807

In [76]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
target_column = 'ProzessData_ActData_AB1_Temperature_DR1_MassMixingStage'

# Normalize features in train_x (excluding the timestamp column)
scaler_x = MinMaxScaler()
train_x_scaled = scaler_x.fit_transform(train_x.iloc[:, 1:])  # Adjust if the first column isn't the timestamp

# Normalize the target column in train_y
scaler_y = MinMaxScaler()
train_y_scaled = scaler_y.fit_transform(train_y[[target_column]])

# Convert to PyTorch tensors
train_x_tensor = torch.tensor(train_x_scaled).float().unsqueeze(1)  # Add time step dimension
train_y_tensor = torch.tensor(train_y_scaled).float()

# DataLoader
train_dataset = TensorDataset(train_x_tensor, train_y_tensor)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

# LSTM model
class LSTMModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(LSTMModel, self).__init__()
        self.lstm = nn.LSTM(input_dim, hidden_dim, batch_first=True)
        self.dropout = nn.Dropout(0.2)
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        lstm_out, _ = self.lstm(x)
        out = self.dropout(lstm_out[:, -1, :])
        out = self.fc(out)
        return out

# Model initialization
input_dim = train_x_tensor.shape[2]  # Number of features
hidden_dim = 50  # Example value
output_dim = train_y_tensor.shape[1]  # Should be 1 as we're predicting a single target

model = LSTMModel(input_dim, hidden_dim, output_dim).to(device)

# Loss and optimizer
criterion = nn.L1Loss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop

num_epochs = 30
for epoch in range(num_epochs):
    total_loss = 0
    total_mae = 0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        
        # Calculate loss (MSE)
        loss = criterion(outputs, labels)
        total_loss += loss.item()

        # Calculate MAE
        #print(type(F.l1_loss(outputs, labels, reduction='sum').item()))
        list1 = outputs.tolist()
        list2 = labels.tolist()
        list1 = np.array(list1).flatten()
        list2 = np.array(list2).flatten()
        abweichung = []
        for a, b in zip(list1, list2):
            
            abweichung.append(abs(a - b))    # Give points
        points = []
        for diff in abweichung:
            if diff < 0.05:
                points.append(1.0)
            elif 0.05 <= diff < 0.1:
                points.append(0.5)
            elif 0.1 <= diff < 0.5:
                points.append(0.25)
            else:
                points.append(0) 

        list1 = points

        points2 = [] 
        consecutive_count =0      
        for i in range(0, len(list1)):
            
            if list1[i] >=0.5 and consecutive_count == 0 :

                points2.append (1)

            elif list1[i] >=0.5 and consecutive_count != 0:

                if consecutive_count == 1:
                    points2.append (.5)
                    points2.append(1)
        
                elif 2 <= consecutive_count <= 10:
                    points2.extend([0.25] * consecutive_count)
                    points2.append(1)
                else:
                    points2.extend([0] * consecutive_count)
                    points2.append(1)

                consecutive_count = 0# Reset consecutive count

            else:
                consecutive_count+=1

        
        if consecutive_count == 1:
            points2.append (.5)
        elif 2 <= consecutive_count <= 0.01:
            points2.extend([0.25] * consecutive_count)
        else:
            points2.extend([0] * consecutive_count)





        total = sum(points)+sum(points2) 
        maxpoint = len(points)*2

        #print(" Punkte aus abweichungslist:", sum(points), "Punkte aus Zeitabweichung:", sum(points2), "max Punkte:",len(points))

        #print(total/maxpoint)
        
        mae = ((maxpoint/total)-1)*100
        total_mae += mae

        # Backward pass and optimize
        loss.backward()
        optimizer.step()

    # Average loss and MAE over the epoch
    avg_loss = total_loss / len(train_loader)
    avg_mae = total_mae / len(train_loader.dataset)

    print(f'Epoch {epoch+1}/{num_epochs}, Loss: {avg_loss:.4f}, MAE: {avg_mae:.4f}')
    print(total/maxpoint)

NameError: name 'y_test' is not defined

In [56]:
def evaluation(X_test, y_test, y_pred, pca=False, examples=True):
    mae = mean_absolute_error(y_test, y_pred)
    print(f"MAE: {mae}")
    
    

    # Calculate absolute differences element-wise
    abweichung = [abs(a - b[0]) for a, b in zip(y_pred, y_test)]

    points = []
    for diff in abweichung:
        if diff < 0.05:
            points.append(1.0)
        elif 0.05 <= diff < 0.1:
            points.append(0.5)
        elif 0.1 <= diff < 0.5:
            points.append(0.25)
        else:
            points.append(0)

    list1 = points

    points2 = []
    consecutive_count = 0
    for i in range(0, len(list1)):
        if list1[i] >= 0.5 and consecutive_count == 0:
            points2.append(1)
        elif list1[i] >= 0.5 and consecutive_count != 0:
            if consecutive_count == 1:
                points2.append(0.5)
                points2.append(1)
            elif 2 <= consecutive_count <= 10:
                points2.extend([0.25] * consecutive_count)
                points2.append(1)
            else:
                points2.extend([0] * consecutive_count)
                points2.append(1)
            consecutive_count = 0  # Reset consecutive count
        else:
            consecutive_count += 1

    if consecutive_count == 1:
        points2.append(0.5)
    elif 2 <= consecutive_count <= 10:
        points2.extend([0.25] * consecutive_count)
    else:
        points2.extend([0] * consecutive_count)

    total = sum(points) + sum(points2)
    maxpoint = len(points) * 2

    print("Punkte aus abweichungslist:", sum(points), "Punkte aus Zeitabweichung:", sum(points2), "max Punkte:", len(points))

    print(total / maxpoint)

    if not pca and examples:
        # Print some example predictions
        example_indices = [0, 1, 2, 3, 4]
        for i in example_indices:
            print(f"Index {i}: | Actual y: {y_test[i][0]} | Predicted y: {y_pred[i]}")

# Call the evaluation function



# Utility Functions
def add_time_features(df):
    df['hour'] = df.index.hour
    df['minute'] = df.index.minute
    daytime_arr = []
    for idx in df.index:
        if df.loc[idx, "hour"] >= 6 and df.loc[idx, "hour"] <= 18:
            daytime_arr.append(1)
        else:
            daytime_arr.append(0)

    df['day_time'] = daytime_arr
    df['night_time'] = 1 - df['day_time']
    return df

In [48]:
# Convert the data into DMatrix format
dtrain = xgb.DMatrix(X1_train, label=y1_train)
dtest = xgb.DMatrix(X1_test, label=y1_test)



# Set the parameters for XGBoost
params = {
    'objective': 'reg:squarederror',
    'eval_metric': 'mae',
    
    'learning_rate': 0.1,
    'max_depth': 5,  # Choose one value, not a list
    'subsample': 0.8,
    'colsample_bytree': 0.8,
    'min_child_weight': 1,  # Choose one value, not a list
    'gamma': 1,
    'alpha': 10
    
}

# Train the XGBoost model with early stopping
num_rounds = 1000
evals = [(dtrain, 'train'), (dtest, 'validation')]
xgb_model = xgb.train(params, dtrain, num_boost_round=num_rounds, evals=evals, early_stopping_rounds=10, verbose_eval=True)



# Save the trained model
xgb_model.save_model('../../data/modeltestphilip.json')



[0]	train-mae:5.38235	validation-mae:5.53709
[1]	train-mae:4.84427	validation-mae:4.98382
[2]	train-mae:4.36000	validation-mae:4.48548
[3]	train-mae:3.92415	validation-mae:4.03757
[4]	train-mae:3.53188	validation-mae:3.63411
[5]	train-mae:3.17882	validation-mae:3.28048
[6]	train-mae:2.86107	validation-mae:2.95330
[7]	train-mae:2.57506	validation-mae:2.65869
[8]	train-mae:2.31768	validation-mae:2.39366
[9]	train-mae:2.08603	validation-mae:2.15519
[10]	train-mae:1.87753	validation-mae:1.94015
[11]	train-mae:1.68988	validation-mae:1.74689
[12]	train-mae:1.52099	validation-mae:1.57283
[13]	train-mae:1.36900	validation-mae:1.41626
[14]	train-mae:1.23222	validation-mae:1.27529
[15]	train-mae:1.10912	validation-mae:1.14848
[16]	train-mae:0.99831	validation-mae:1.03465
[17]	train-mae:0.89858	validation-mae:0.93218
[18]	train-mae:0.80884	validation-mae:0.83949
[19]	train-mae:0.72807	validation-mae:0.75621
[20]	train-mae:0.65539	validation-mae:0.68132
[21]	train-mae:0.58997	validation-mae:0.6141

In [63]:

dtrain2 = xgb.DMatrix(X2_train, label=y2_train)
#dtest2 = xgb.DMatrix(X2_test, label=y2_test)

dtrain3 = xgb.DMatrix(X3_train, label=y3_train)
dtest3 = xgb.DMatrix(X3_test, label=y3_test)

existing_model = xgb.Booster(model_file='../../data/modeltestphilip.json')
params = {
    'objective': 'reg:squarederror',
    'eval_metric': 'mae',
    'learning_rate': 0.01,
    'max_depth': 3,
    'subsample': 0.8,
    'colsample_bytree': 0.8,
    'min_child_weight': 1,
    'gamma': 1,
    'alpha': 0.1
}

# Continue training the existing model on new data
num_boost_round = 300
xgb_model = xgb.train(params, dtrain2, num_boost_round=num_boost_round, xgb_model=existing_model)
num_boost_round = 300
xgb_model = xgb.train(params, dtrain3, num_boost_round=num_boost_round, xgb_model=existing_model)

# Save the updated model
xgb_model.save_model('../../data/modeltestxgb.model')

# Make predictions on the validation data
y_pred_val = xgb_model.predict(dtest3)

# Convert NumPy array to Python list
y_pred_val_list = y_pred_val.tolist()
y_pred_val_list = [float(d) for d in y_pred_val_list]
print(type(y_pred_val_list))
print(type(y3_test))

y3_test_list = y3_test.values.tolist()


# Now you can use X2_train and y_pred_val_list in your further processing or evaluation

# Assuming you have an evaluation function named 'evaluation'
evaluation(X3_train, y3_test_list, y_pred_val_list)
xgb_model.save_model('../../data/modeltestphilip.json')


/home/codespace/.python/current/lib/python3.10/site-packages/xgboost/core.py:160: UserWarning: [06:27:17] WARNING: /workspace/src/c_api/c_api.cc:1240: Saving into deprecated binary model format, please consider using `json` or `ubj`. Model format will default to JSON in XGBoost 2.2 if not specified.
  warnings.warn(smsg, UserWarning)


<class 'list'>
<class 'pandas.core.frame.DataFrame'>
MAE: 0.022771084222007087
Punkte aus abweichungslist: 188994.5 Punkte aus Zeitabweichung: 201586.0 max Punkte: 205203
0.9516929577052967
Index 0: | Actual y: 41.5 | Predicted y: 41.51360321044922
Index 1: | Actual y: 41.5 | Predicted y: 41.51360321044922
Index 2: | Actual y: 41.5 | Predicted y: 41.51360321044922
Index 3: | Actual y: 41.5 | Predicted y: 41.51360321044922
Index 4: | Actual y: 41.5 | Predicted y: 41.51360321044922


In [ ]:


def preprocess_data(train_x, train_y, test_x, test_y, target_column):
    # Normalize features in train_x (excluding the timestamp column)
    scaler_x = MinMaxScaler()
    train_x_scaled = scaler_x.fit_transform(train_x.iloc[:, 1:])  # Adjust if the first column isn't the timestamp

    # Flatten time series sequences into feature vectors
    train_x_flattened = train_x_scaled.reshape((len(train_x), -1))

    # Normalize the target column in train_y
    scaler_y = MinMaxScaler()
    train_y_scaled = scaler_y.fit_transform(train_y[[target_column]])

    # Convert to NumPy arrays
    train_x_tensor = np.float32(train_x_flattened)  # No need to add a time step dimension
    train_y_tensor = np.float32(train_y_scaled)

    return train_x_tensor, train_y_tensor


def train_xgboost_model(x_train, y_train):
    # XGBoost Model
    xgboost_model = XGBRegressor(objective='reg:squarederror')
    xgboost_model.fit(x_train, y_train)

    return xgboost_model

def evaluate_model(model, x_test, y_test):
    # Predictions on the test set
    y_pred = model.predict(x_test)

    # Calculate Mean Absolute Error (MAE)
    mae = mean_absolute_error(y_test, y_pred)
    
    return mae

if __name__ == "__main__":
    # Target column
    target_column = 'ProzessData_ActData_AB1_Temperature_DR1_MassMixingStage'

    # Preprocess data
    train_x_tensor, train_y_tensor = preprocess_data(train_x, train_y, test_x, test_y, target_column)

    # Train XGBoost model
    xgboost_model = train_xgboost_model(train_x_tensor, train_y_tensor)

    # Evaluate the XGBoost model
    x_test_tensor = np.float32(test_x.iloc[:, 1:].values[:, :, None])
    y_test_xgboost = test_y[target_column].values
    shape = x_test_tensor.shape

    x_test_tensor = x_test_tensor.reshape((shape[0], shape[1]))
    print("Shape of x_test_tensor:", x_test_tensor.shape)
    mae_xgboost = evaluate_model(xgboost_model, x_test_tensor, y_test_xgboost)

    print(f'XGBoost MAE: {mae_xgboost:.4f}')


NameError: name 'train_x' is not defined

In [ ]:
import torch

# Assuming your model is named 'model'
torch.save(model, '../../models/lstm_philip2.pth')

In [ ]:
with open('../../models/lstm_philip2.pkl', 'wb') as file:
    pickle.dump(model, file)

NameError: name 'pickle' is not defined